# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,pankrushikha,53.8319,80.3406,0.42,90,100,2.63,RU,1730346326
1,1,kharakvasla,18.4333,73.7667,26.30,58,6,2.87,IN,1730346326
2,2,lensk,60.7253,114.9278,-7.52,89,97,1.00,RU,1730346326
3,3,waitangi,-43.9535,-176.5597,13.79,81,39,3.13,NZ,1730346326
4,4,tagiura,32.8817,13.3506,21.19,71,22,5.43,LY,1730346327


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [12]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    x='Lng',        # Update with the actual column name for longitude
    y='Lat',        # Update with the actual column name for latitude
    size='Humidity',  # Replace with the actual column name for humidity
    geo=True,       # Enable geographic plotting
    tiles='OSM',    # Use OpenStreetMap tiles
    color='blue',   
    title='City Humidity Map')

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [15]:
# Narrow down cities that fit criteria and drop any results with null values
filtered_cities = city_data_df[
    (city_data_df['Max Temp'] >= 20) & (city_data_df['Max Temp'] <= 25) & 
    (city_data_df['Humidity'] < 50)  
]
# Drop any rows with null values
filtered_cities = filtered_cities.dropna()

# Display sample data
filtered_cities

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
119,119,pala,9.3642,14.9046,20.59,29,1,1.94,TD,1730346363
147,147,necochea,-38.5473,-58.7368,21.73,47,99,6.38,AR,1730346302
149,149,guaruja,-23.9931,-46.2564,20.50,26,1,2.82,BR,1730346373
227,227,ghadamis,30.1337,9.5007,24.30,25,98,3.56,LY,1730346397
260,260,moussoro,13.6408,16.4901,22.33,20,1,4.73,TD,1730346408
263,263,adre,13.4667,22.2000,20.12,25,4,5.00,TD,1730346409
289,289,qiancheng,27.1850,109.7654,23.66,38,0,3.11,CN,1730346418
293,293,faya-largeau,17.9257,19.1043,20.53,30,6,8.36,TD,1730346419
381,381,zhongxiang,31.1661,112.5831,23.26,42,0,6.24,CN,1730346448
396,396,liuduzhai,27.3661,110.9350,22.73,38,0,2.73,CN,1730346454


### Step 3: Create a new DataFrame called `hotel_df`.

In [19]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = filtered_cities[['City','Country','Lat', 'Lng','Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''  # Initialize the column with empty strings

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
119,pala,TD,9.3642,14.9046,29,
147,necochea,AR,-38.5473,-58.7368,47,
149,guaruja,BR,-23.9931,-46.2564,26,
227,ghadamis,LY,30.1337,9.5007,25,
260,moussoro,TD,13.6408,16.4901,20,
263,adre,TD,13.4667,22.2000,25,
289,qiancheng,CN,27.1850,109.7654,38,
293,faya-largeau,TD,17.9257,19.1043,30,
381,zhongxiang,CN,31.1661,112.5831,42,
396,liuduzhai,CN,27.3661,110.9350,38,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10,000
params = {
    # YOUR CODE HERE
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    # YOUR CODE HERE

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = # YOUR CODE HERE
    params["bias"] = # YOUR CODE HERE

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = # YOUR CODE HERE

    # Convert the API response to JSON format
    name_address = # YOUR CODE HERE

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
saint-pierre - nearest hotel: Tropic Hotel
badger - nearest hotel: Clarion Hotel & Suites
ust'-kulom - nearest hotel: No hotel found
emerald - nearest hotel: Emerald Central Hotel
newman - nearest hotel: No hotel found
fort bragg - nearest hotel: Airborne Inn Lodging
laguna - nearest hotel: Holiday Inn Express & Suites
nova vicosa - nearest hotel: Pousada Pontal da Barra
fort myers shores - nearest hotel: Comfort Inn Fort Myers
trinity beach - nearest hotel: Blue Lagoon Resort
racine - nearest hotel: No hotel found
six-fours-les-plages - nearest hotel: Kyriad Direct La Seyne-sur-Mer
porto velho - nearest hotel: Hotel Slaviero Essential Porto Velho
posto fiscal rolim de moura - nearest hotel: No hotel found
port douglas - nearest hotel: Mantra Aqueous Port Douglas
roswell - nearest hotel: Best Western Plus Hotel
hell-ville - nearest hotel: ABUD HOTEL
sona - nearest hotel: Al Bosco
morehead city - nearest hotel: Bask Hotel Big Rock Landing, Trademark Collection by W

,City,Country,Lat,Lng,Humidity,Hotel Name
43,saint-pierre,RE,-21.3393,55.4781,61,Tropic Hotel
66,badger,US,64.8000,-147.5333,37,Clarion Hotel & Suites
102,ust'-kulom,RU,61.6864,53.6902,43,No hotel found
120,emerald,AU,-23.5333,148.1667,28,Emerald Central Hotel
130,newman,US,37.3138,-121.0208,51,No hotel found
138,fort bragg,US,35.1390,-79.0060,80,Airborne Inn Lodging
146,laguna,US,38.4210,-121.4238,52,Holiday Inn Express & Suites
160,nova vicosa,BR,-17.8919,-39.3719,87,Pousada Pontal da Barra
170,fort myers shores,US,26.7092,-81.7459,95,Comfort Inn Fort Myers
176,trinity beach,AU,-16.7888,145.6968,44,Blue Lagoon Resort


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [21]:
%%capture --no-display

# Configure the map plot
hotel_map_plot = hotel_df.hvplot.points(
    'Lng', 
    'Lat', 
    geo=True, 
    tiles='EsriImagery', 
    frame_width=800, 
    frame_height=600, 
    size='Humidity',
    scale=0.5, 
    color='City', 
    hover_cols=['Hotel Name', 'Country'])

# Display the map
hotel_map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)